# DWM PROJECT 2021
### John Coast - 880892

Index:
- [Analisi del dataset "Train"](#analisi_train)
- [Analisi del dataset "Properties"](#analisi_prop)
- [Features Engineering](#features_engineer)
- [Gestione dei missing values e rimozione delle colonne non necessarie o che presentano multicollinearità](#missing_val)
- [Recupero di missing values per features connesse a posizione geografica e tassazione](#recover_missing_pos_tax)
- [Aggiunta di features custom potenzialmente utili](#custom_features)
- [Features importance, features selection e preparazione del dataset finale](#features_selection)
- [Corstruzione del modello sfruttante Random Forest e tuning dei suoi parametri](#mod_1)
- [Corstruzione del modello sfruttante Gradient Boosting e tuning dei suoi parametri](#mod_2)
- [Comparazione e analisi dei modelli](#comparison_and_analisys)
- [Considerazioni finali](#final)

In [ ]:
# Library imports
%matplotlib inline
%load_ext autoreload
%autoreload 2

from utils.general_utils import *
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings, random, joblib, os
import xgboost as xgb
from sklearn.utils import resample
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import root_mean_squared_error
from sklearn.feature_selection import RFECV
from sklearn.model_selection import GridSearchCV

# Set up the working directory
INPUT_DATA_DIR = "../data/input"
OUTPUT_DATA_DIR = "../data/output"
MODEL_DIR = "models"
PROPS_FILENAME = "properties_2016.parquet"
TRAIN_DATA_FILENAME = "train_2016_v2.csv"
PROPS_PATH = os.path.join(INPUT_DATA_DIR, PROPS_FILENAME)
TRAIN_DATA_PATH = os.path.join(INPUT_DATA_DIR, TRAIN_DATA_FILENAME)
SEED = 42
TOP_K_DISPLAY = 20

# Set up the random seed
np.random.seed(SEED)
random.seed(SEED)
# Display options
pd.options.display.float_format = '{:.4f}'.format

<a id='analisi_train'></a>
### Analysis of the "Train" dataset

In [ ]:
df_train = pd.read_csv(
    TRAIN_DATA_PATH, parse_dates=["transactiondate"], date_format="%Y-%m-%d"
)
df_train[df_train.select_dtypes(np.float64).columns] = df_train.select_dtypes(
    np.float64
).astype(np.float32)
df_train.info(max_cols=TOP_K_DISPLAY)
print("Shape: ", df_train.shape)

In [ ]:
# Missing value ratio
df_train.isna().sum() / df_train.shape[0] * 100

We note that the dataset contains 3 columns:
- Parcelid: unique ID that identifies each building instance
- Logerror: index that we should use to verify the goodness of our model<br>
From the competition website: *"Zillow is asking you to predict the log-error between their Zestimate and the actual sale price, given all the features of a home. The log error is defined as logerror=log(Zestimate)−log( SalePrice)"*
- Transactiondate: Actual or estimated sale date for that building instance

Also there are no null values ​​in the dataset but there are duplicate parcelids, even though the parcelid and transactiondate pair is unique for each row. This implies that there are actual (or forecasted) sales data for the same building on different days

<a id='analysis_prop'></a>
### Analysis of the "Properties" dataset

In [ ]:
df_prop = pd.DataFrame([])
if PROPS_PATH.endswith(".csv"):
    df_prop = pd.read_csv(PROPS_PATH)
    to_float64_float32(df_prop)
    df_prop.to_parquet(
        os.path.join(INPUT_DATA_DIR, PROPS_FILENAME.split(".")[0] + ".parquet")
    )
else:
    df_prop = pd.read_parquet(PROPS_PATH)

# df_prop=pd.read_csv(input_folder+data_file_name, header=0) #parameter nrows=x to limit the number of rows

df_prop.info(max_cols=TOP_K_DISPLAY)
print("Shape: ", df_prop.shape)

In [ ]:
# Missing value ratio
df_prop.isna().sum().sort_values(ascending=False)[:TOP_K_DISPLAY] / df_prop.shape[
    0
] * 100

In [ ]:
# Missing values ​​plot
df_missing = df_prop.isnull().sum(axis=0).reset_index()
df_missing.columns = ["column_name", "missing_count"]
df_missing = df_missing.loc[df_missing["missing_count"] > 0]
df_missing = df_missing.sort_values(by="missing_count")

ind = np.arange(df_missing.shape[0])
fig, ax = plt.subplots(figsize=(12, 18))
rects = ax.barh(ind, df_missing["missing_count"])
ax.set_yticks(ind)
ax.set_yticklabels(df_missing["column_name"], rotation="horizontal")
ax.set_xlabel("Count of missing values")
ax.set_title("Number of missing values in each column")
plt.show()
del df_missing

In [114]:
# Columns containing categorical variables based on the documentation provided on kaggle.com (used later)
other_cols = [
    "parcelid",
    "airconditioningtypeid",
    "architecturalstyletypeid",
    "buildingqualitytypeid",
    "buildingclasstypeid",
    "decktypeid",
    "fips",
    "fireplaceflag",
    "hashottuborspa",
    "heatingorsystemtypeid",
    "pooltypeid10",
    "pooltypeid2",
    "pooltypeid7",
    "propertycountylandusecode",
    "propertylandusetypeid",
    "propertyzoningdesc",
    "rawcensustractandblock",
    "censustractandblock",
    "regionidcounty",
    "regionidcity",
    "regionidzip",
    "regionidneighborhood",
    "regionidzip",
    "storytypeid",
    "typeconstructiontypeid",
    "assessmentyear",
    "taxdelinquencyflag",
    "taxdelinquencyyear",
    "yearbuilt",
]
numerical_cols = [x for x in df_prop.columns if x not in other_cols]

In [ ]:
# Displaying column values ​​of non-float, int, or datetime type
not_float_col = df_prop.select_dtypes(exclude=[np.float32, np.int64]).columns
for c in not_float_col:
    print("Column: " + c)
    print("values: ", df_prop[c].unique()[:TOP_K_DISPLAY], "\n")

The dataset has many features and instances:
- some of these having type float32 are actually numeric identifiers of integer type associated with categories described in more detail in the Kaggle.com documentation
- those of type object instead are strings representing boolean values ​​or codes relating to categories

The various columns will then be managed subsequently to represent the correct semantics of the data.

The dataset also does not present duplicates either in parcelid or globally, but it contains a very high number of missing values: in fact, several columns present a missing ratio higher than 97%. In the following chapters, it will be illustrated how these missing values ​​are managed for each of the features present.

For this particular supervised learning task, only the houses inside the "train_2016_v2" file are sufficient and not all those present in "properties_2016", so we proceed to merge the two datasets previously analyzed in order to have a single one containing the set of all the columns and only the rows that will actually be used for the prediction.

In [ ]:
df_prop.info(max_cols=TOP_K_DISPLAY)

Dataset after merge:

In [ ]:
df_prop = df_train.merge(df_prop, on="parcelid")
df_prop.info(max_cols=TOP_K_DISPLAY)
print("Shape: ", df_prop.shape)

<a id='features_engineer'></a>
## Features Engineering

<a id='missing_val'></a>
### Handling missing values ​​and removing unnecessary or multicollinear columns

In [ ]:
# Heatmap to visualize correlations between variables
plt.figure(figsize=(12, 8))
sns.heatmap(data=df_prop[numerical_cols].corr(), cmap="Reds")
plt.show()
plt.gcf().clear()

From the analysis of the heatmap on the correlation between the numerical variables, it is noted that the features 'calculatedfinishedsquarefeet', 'finishedsquarefeet12', 'finishedsquarefeet13', 'finishedsquarefeet15' and 'finishedsquarefeet6' are highly correlated being dark red in color.
The same consideration also applies to 'finishedfloor1squarefeet' and 'finishedsquarefeet50' which additionally have the same description in the documentation provided.
Similarly 'bathroomcnt', 'calculatedbathnbr' and 'fullbathcnt' are also related.

Only 'calculatedfinishedsquarefeet' is kept among the four because it has fewer missing values, while between 'finishedfloor1squarefeet' and 'finishedsquarefeet50' 'finishedsquarefeet50' is arbitrarily removed because the missing ratio is equivalent.

'bathroomcnt' and 'calculatedbathnbr' are also removed, leaving 'bathroomcnt', as a similar intuition to the previous one is applied

In [ ]:
df_prop[
    [
        "calculatedfinishedsquarefeet",
        "finishedsquarefeet12",
        "finishedsquarefeet13",
        "finishedsquarefeet15",
        "finishedsquarefeet6",
    ]
].isna().sum() / df_prop.shape[0] * 100

In [120]:
df_prop.drop(
    columns=[
        "finishedsquarefeet12",
        "finishedsquarefeet13",
        "finishedsquarefeet15",
        "finishedsquarefeet6",
    ],
    inplace=True,
)

In [ ]:
df_prop[
    ["finishedfloor1squarefeet", "finishedsquarefeet50"]
].isna().sum() / df_prop.shape[0] * 100

In [122]:
df_prop.drop(columns="finishedsquarefeet50", inplace=True)

In [ ]:
df_prop[
    ["calculatedbathnbr", "bathroomcnt", "fullbathcnt"]
].isna().sum() / df_prop.shape[0] * 100

In [124]:
df_prop.drop(columns=["calculatedbathnbr", "fullbathcnt"], inplace=True)

'hashottuborspa' and 'pooltypeid10' have semantically the same description. It is decided to remove 'pooltypeid10' which has fewer missing values.

It is also assumed that if the pool/hot tub value (features 'pooltypeid2', 'pooltypeid7', 'poolcnt') is not present this indicates 0 elements present in the building

In [ ]:
print(df_prop["hashottuborspa"].value_counts())
print(df_prop["pooltypeid10"].value_counts())

In [ ]:
df_prop[["hashottuborspa", "pooltypeid10"]].isnull().sum() / df_prop.shape[0] * 100

In [127]:
df_prop.drop(columns="pooltypeid10", inplace=True)

In [128]:
df_prop[["pooltypeid2", "pooltypeid7", "poolcnt"]] = df_prop[
    ["pooltypeid2", "pooltypeid7", "poolcnt"]
].fillna(0)
df_prop["hashottuborspa"] = df_prop["hashottuborspa"].fillna(0)  # >90% na
df_prop["hashottuborspa"] = df_prop["hashottuborspa"].astype(bool);

For the 'poolsizesum' feature, the median of the values ​​present in the rows where the 'poolcnt' column is greater than 0 is used, otherwise this value is set to 0
(we chose to use the median as a filler because it is less influenced by outliers and because we assume that the dimensions of swimming pools in the United States are more or less standard)

In [129]:
median_poolsize = df_prop[df_prop["poolcnt"] > 0]["poolsizesum"].median()
df_prop.loc[
    (df_prop["poolcnt"] > 0) & (df_prop["poolsizesum"].isna()), "poolsizesum"
] = median_poolsize

# If you don't have a pool, the pool size is 0
df_prop.loc[(df_prop["poolcnt"] == 0), "poolsizesum"] = 0

'fireplaceflag' and 'fireplacecnt' have inconsistencies:
as you can see from the analysis below there are lines where 'fireplacecnt' is present, while 'fireplaceflag' is missing or incorrect.

We then proceed by setting the values ​​of 'fireplacecnt' to 0 when NaN and setting 'fireplaceflag' to True when 'fireplacecnt' is present, False otherwise.

In [130]:
df_prop["fireplaceflag"] = df_prop["fireplaceflag"].fillna(0)  # >90% na
df_prop["fireplaceflag"] = df_prop["fireplaceflag"].astype(bool)
df_prop.loc[(~df_prop["fireplacecnt"].isna()), "fireplaceflag"] = True
df_prop["fireplacecnt"] = df_prop["fireplacecnt"].fillna(0)

The values ​​of 'taxdelinquencyflag', 'garagecarcnt', 'garagetotalsqft' are filled assuming 0 if the value is NaN, as done previously

In [131]:
df_prop[["taxdelinquencyflag", "garagecarcnt", "garagetotalsqft"]] = df_prop[
    ["taxdelinquencyflag", "garagecarcnt", "garagetotalsqft"]
].fillna(0)
df_prop["taxdelinquencyflag"] = df_prop["taxdelinquencyflag"].astype(bool)

Features like 'airconditioningtypeid', 'heatingorsystemtypeid', 'threequarterbathnbr' instead are considered unimportant and variable so it is decided to replace the missing values ​​of these categorical features with their respective mode.<br>
As can be seen from the graphs below, the frequency of these categorical variables is intuitively correct, it is reasonably assumed that AC and Heating System are more commonly 'Central' and that most homes only have a three-quarter bathroom.

In [ ]:
plt.figure(figsize=(8, 5))
df_prop["airconditioningtypeid"].astype(int, errors="ignore").hist(grid=False)
plt.show()

mode = float(df_prop["airconditioningtypeid"].mode()[0])
print("Moda: ", mode)
df_prop["airconditioningtypeid"] = df_prop["airconditioningtypeid"].fillna(mode)

In [ ]:
plt.figure(figsize=(10, 6))
df_prop["heatingorsystemtypeid"].astype(int, errors="ignore").hist(grid=False)
plt.show()

mode = float(df_prop["heatingorsystemtypeid"].mode()[0])
print("Moda: ", mode)
df_prop["heatingorsystemtypeid"] = df_prop["heatingorsystemtypeid"].fillna(mode)

In [ ]:
plt.figure(figsize=(5, 5))
df_prop["threequarterbathnbr"].astype(int, errors="ignore").hist(grid=False)
plt.show()

mode = float(df_prop["threequarterbathnbr"].mode()[0])
print("Moda: ", mode)
df_prop["threequarterbathnbr"] = df_prop["threequarterbathnbr"].fillna(mode)

Finally, it was decided to remove the features with a missing ratio greater than 97% because they were considered to have too little information to be useful for the regression task.

In [135]:
tmp_col = df_prop.columns
for c in tmp_col:
    if df_prop[c].isna().sum() / df_prop.shape[0] > 0.97:
        df_prop.drop(columns=c, inplace=True)

In [ ]:
tmp_col = df_prop.columns
tmp_list = []

for c in tmp_col:
    if df_prop[c].isna().sum() / df_prop.shape[0] > 0.0:
        tmp_list.append(c)

print("Number of features with missing values: ", len(tmp_list))
print(tmp_list)
print("Number of total features after droping: ", len(df_prop.columns))

The remaining features are then analyzed, trying to restore the last missing values.

<a id='recover_missing_pos_tax'></a>
### Recovering missing values ​​for features related to geographic location and taxation


In [ ]:
geo_col_names = [
    "latitude",
    "longitude",
    "buildingqualitytypeid",
    "propertycountylandusecode",
    "propertyzoningdesc",
    "regionidcity",
    "regionidneighborhood",
    "regionidzip",
    "unitcnt",
    "yearbuilt",
]
df_geo=df_prop[geo_col_names]
df_geo.isna().sum() / df_geo.shape[0] * 100

Intuitively, given the absence of missing values, the 'latitude' and 'longitude' features relating to the position of the building could be used to recover other attributes that are not present: geographically close houses are in fact thought to have certain similar characteristics<br><br>
The original values ​​of these two features are then restored because, as described in the documentation provided, those within the dataset are multiplied by 10^6.<br>

In [138]:
df_prop["latitude"] = df_prop["latitude"] / (10**6)
df_prop["longitude"] = df_prop["longitude"] / (10**6)
# df_prop.dropna( axis = 0, subset = ['latitude', 'longitude'], inplace = True )

It was decided to use the K-nearest neighbors (KNN) algorithm to perform the recovery task because it was considered the most suitable given that it is based on learning through analogies between neighboring instances and given its computational efficiency. The code cell below also explains the motivations and hypotheses used for the recovery of the categorical and numerical variables

In [ ]:
df_prop["buildingqualitytypeid"]

In [ ]:
warnings.simplefilter(action='ignore', category=UserWarning)
parameters = {"n_neighbors": [1, 2, 3, 4, 5, 8, 10]}


# It is assumed that blocks of houses close by were all built more or less at the same time and therefore have similar quality.
fillna_knn(
    df=df_prop,
    base=["latitude", "longitude"],
    target="buildingqualitytypeid",
    tuning_params=parameters,
)

# Neighboring homes have the same countrylandusecode
tmp_label_enc = zoningcode2int(df=df_prop, target="propertycountylandusecode")
fillna_knn(
    df=df_prop,
    base=["latitude", "longitude"],
    target="propertycountylandusecode",
    tuning_params=parameters,
)
df_prop["propertycountylandusecode"] = tmp_label_enc.inverse_transform(
    df_prop["propertycountylandusecode"].astype(int)
)


# Neighboring homes have the same zoning description
tmp_label_enc = zoningcode2int(df=df_prop, target="propertyzoningdesc")
fillna_knn(
    df=df_prop,
    base=["latitude", "longitude"],
    target="propertyzoningdesc",
    tuning_params=parameters,
)
df_prop["propertyzoningdesc"] = tmp_label_enc.inverse_transform(
    df_prop["propertyzoningdesc"].astype(int)
)


# Here too, neighboring properties are assumed to have the same regionedcity, regionedneighborhood and regionedzip
fillna_knn(
    df=df_prop,
    base=["latitude", "longitude"],
    target="regionidcity",
    tuning_params=parameters,
)

fillna_knn(
    df=df_prop,
    base=["latitude", "longitude"],
    target="regionidneighborhood",
    tuning_params=parameters,
)

fillna_knn(
    df=df_prop,
    base=["latitude", "longitude"],
    target="regionidzip",
    tuning_params=parameters,
)


# Same intuition for the fields 'unitcnt' (Number of units the structure is built into), 'yearbuilt' and 'lotsizesquarefeet'
fillna_knn(
    df=df_prop,
    base=["latitude", "longitude"],
    target="unitcnt",
    tuning_params=parameters,
)

fillna_knn(
    df=df_prop,
    base=["latitude", "longitude"],
    target="yearbuilt",
    tuning_params=parameters,
)

fillna_knn_reg(
    df=df_prop,
    base=["latitude", "longitude"],
    target="lotsizesquarefeet",
    tuning_params=parameters,
)

warnings.simplefilter(action='default', category=UserWarning)

As for the feature 'finishedfloor1squarefeet', as you can also see from the heatmap at the beginning of the current section, this is correlated with 'calculatedfinishedsquarefeet'. We therefore try to use the latter to fill its values

In [ ]:
plt.figure(figsize=(13, 13))
sns.jointplot(
    x=df_prop["finishedfloor1squarefeet"].values,
    y=df_prop["calculatedfinishedsquarefeet"].values,
)
plt.ylabel("calculatedfinishedsquarefeet", fontsize=10)
plt.xlabel("finishedfloor1squarefeet", fontsize=10)
plt.title("finishedfloor1squarefeet Vs calculatedfinishedsquarefeet", fontsize=13)
plt.show()

From the graph, we can see that in some houses the values ​​of the features are exactly the same: probably some houses have their total area occupied by only one room (as could be a sort of study). This information is therefore assumed to be true and the filling is carried out.<br>
Furthermore, some rows of the dataset contain values ​​of 'finishedfloor1squarefeet' greater than the total size of the house, probably due to an incorrect input data entry. It is therefore decided to remove these rows from the dataset

In [142]:
df_prop.loc[
    (df_prop["finishedfloor1squarefeet"].isna()) & (df_prop["numberofstories"] == 1),
    "finishedfloor1squarefeet",
] = df_prop.loc[
    (df_prop["finishedfloor1squarefeet"].isna()) & (df_prop["numberofstories"] == 1),
    "calculatedfinishedsquarefeet",
]

droprows = df_prop.loc[
    df_prop["calculatedfinishedsquarefeet"] < df_prop["finishedfloor1squarefeet"]
].index
df_prop = df_prop.drop(droprows)

In [ ]:
tmp_col = df_prop.columns
tmp_list = []

for c in tmp_col:
    if df_prop[c].isna().sum() / df_prop.shape[0] > 0.0:
        tmp_list.append(c)

print("Number of features with missing values: ", len(tmp_list))
print(tmp_list)

Now we are going to handle the variables related to building taxes: in particular we are trying to retrieve the values ​​of 'structuretaxvaluedollarcnt', 'taxamount' and 'landtaxvaluedollarcnt'.
The variable 'taxvaluedollarcnt' is assumed to be the most significant to use as a support since it also contains the least number of missing values.<br>

The NaN values ​​of 'taxvaluedollarcnt' are then filled using its median, in order to have a result less sensitive to outliers.

In [ ]:
tax_col_names = [
    "taxvaluedollarcnt",
    "landtaxvaluedollarcnt",
    "structuretaxvaluedollarcnt",
    "taxamount",
]
df_tax = df_prop[tax_col_names]
df_tax.isna().sum() / df_tax.shape[0] * 100

In [145]:
median = df_prop["taxvaluedollarcnt"].median()
df_prop["taxvaluedollarcnt"] = df_prop["taxvaluedollarcnt"].fillna(median)

From correlation analysis and distribution graphs for the three target variables, it is also noted that 'taxvaluedollarcnt' is the most correlated variable for all of them: we therefore try to perform a prediction of the missing values ​​using the KNN algorithm for the features 'structuretaxvaluedollarcnt', 'taxamount' and 'landtaxvaluedollarcnt'

In [ ]:
x = df_tax.corr()

print("Target: structuretaxvaluedollarcnt")
print(x["structuretaxvaluedollarcnt"].sort_values(ascending=False))
plt.figure(figsize=(12, 12))
sns.jointplot(
    x=df_tax["structuretaxvaluedollarcnt"].values, y=df_tax["taxvaluedollarcnt"].values
)
plt.ylabel("taxvaluedollarcnt", fontsize=12)
plt.xlabel("structuretaxvaluedollarcnt", fontsize=12)
plt.title("structuretaxvaluedollarcnt Vs taxvaluedollarcnt", fontsize=15)
plt.show()

print("Target: taxamount")
print(x["taxamount"].sort_values(ascending=False))
plt.figure(figsize=(12, 12))
sns.jointplot(x=df_tax["taxamount"].values, y=df_tax["taxvaluedollarcnt"].values)
plt.ylabel("taxvaluedollarcnt", fontsize=12)
plt.xlabel("taxamount", fontsize=12)
plt.title("taxamount Vs taxvaluedollarcnt", fontsize=15)
plt.show()

print("Target: landtaxvaluedollarcnt")
print(x["landtaxvaluedollarcnt"].sort_values(ascending=False))
plt.figure(figsize=(12, 12))
sns.jointplot(
    x=df_tax["landtaxvaluedollarcnt"].values, y=df_tax["taxvaluedollarcnt"].values
)
plt.ylabel("taxvaluedollarcnt", fontsize=12)
plt.xlabel("landtaxvaluedollarcnt", fontsize=12)
plt.title("landtaxvaluedollarcnt Vs taxvaluedollarcnt", fontsize=15)
plt.show()

In [ ]:
parameters = {"n_neighbors": [10, 20, 30, 40, 50, 100]}
fillna_knn_reg(
    df=df_prop,
    base=["taxvaluedollarcnt"],
    target="structuretaxvaluedollarcnt",
    tuning_params=parameters,
)
fillna_knn_reg(
    df=df_prop, base=["taxvaluedollarcnt"], target="taxamount", tuning_params=parameters
)
fillna_knn_reg(
    df=df_prop,
    base=["taxvaluedollarcnt"],
    target="landtaxvaluedollarcnt",
    tuning_params=parameters,
)

In [ ]:
tmp_col = df_prop.columns
tmp_list = []

for c in tmp_col:
    if df_prop[c].isna().sum() / df_prop.shape[0] > 0.0:
        tmp_list.append(c)

print("Number of features with missing values: ", len(tmp_list))
print(tmp_list)

As you can see, there are only a few features left with missing values, so we proceed:
- filling 'numberofstories' with her fashion
- removing the 'censustractandblock' column because it is assumed that the 'rawcensustractandblock' attribute contains the same information albeit in a less elaborate way.
- building a predictor for 'calculatedfinishedsquarefeet' based on 'bathroomcnt', 'bedroomcnt', 'structuretaxvaluedollarcnt'. The total size of the living area is assumed to depend on how many bathrooms and bedrooms there are and how much structure tax is attributed to the dwelling.
- building a predictor for 'finishedfloor1squarefeet' based on the same features as above and the number of stories in the building (numberofstories). It is assumed that the number of stories present can be useful to calculate the feature 'finishedfloor1squarefeet' (*Dimensions of the finished living space on the first floor (entrance) of the house*)

In [ ]:
plt.figure(figsize=(5, 5))
x_min = df_prop["numberofstories"].min()
x_max = df_prop["numberofstories"].max()
plt.xlim([x_min, x_max])
plt.xticks(np.arange(x_min, x_max + 1, 1))
df_prop["numberofstories"].hist()
plt.grid()
plt.show()

mode = float(df_prop["numberofstories"].mode()[0])
print("Moda: ", mode)
df_prop["numberofstories"] = df_prop["numberofstories"].fillna(mode)

In [150]:
df_prop.drop(columns="censustractandblock", inplace=True)

In [ ]:
fillna_knn_reg(
    df=df_prop,
    base=["bathroomcnt", "bedroomcnt", "structuretaxvaluedollarcnt"],
    target="calculatedfinishedsquarefeet",
    tuning_params=parameters,
)
fillna_knn_reg(
    df=df_prop,
    base=["bathroomcnt", "bedroomcnt", "structuretaxvaluedollarcnt", "numberofstories"],
    target="finishedfloor1squarefeet",
    tuning_params=parameters,
)

In [ ]:
tmp_col = df_prop.columns
tmp_list = []

for c in tmp_col:
    if df_prop[c].isna().sum() / df_prop.shape[0] > 0.0:
        tmp_list.append(c)

print("Number of missing features: ", len(tmp_list))

In [ ]:
print("Columns:")
print(df_prop.columns)
print("Shape: ", df_prop.shape)

<a id='custom_features'></a>
### Adding potentially useful custom features

Some custom features are now created and added to the dataset that could intuitively be useful for building the final model and to better explain the data trend.

##### Features related to building properties

In [154]:
# Age of the building at the time of sale
df_prop["yearbuilt"] = pd.to_datetime(df_prop["yearbuilt"], format="%Y")
df_prop["assessmentyear"] = pd.to_datetime(df_prop["assessmentyear"], format="%Y")
df_prop["Life-until-selling"] = (
    df_prop["transactiondate"] - df_prop["yearbuilt"]
).dt.days

# Relationship between structure value and land area
df_prop["N-ValueProp"] = (
    df_prop["structuretaxvaluedollarcnt"] / df_prop["landtaxvaluedollarcnt"]
)

# Portion of livable area
df_prop["N-LivingAreaProp"] = (
    df_prop["calculatedfinishedsquarefeet"] / df_prop["lotsizesquarefeet"]
)

# Amount of extra space
df_prop["N-ExtraSpace"] = (
    df_prop["lotsizesquarefeet"] - df_prop["calculatedfinishedsquarefeet"]
)

# Features that indicate whether the property has a garage, pool, or jacuzzi and AC
df_prop["N-GarPoolAC"] = (
    (df_prop["garagecarcnt"] > 0)
    & (df_prop["hashottuborspa"] > 0)
    & (df_prop["airconditioningtypeid"] != 5)
) * 1
df_prop["N-GarPoolAC"] = df_prop["N-GarPoolAC"].astype(bool)

In [155]:
# Home Tax to Total Tax Ratio by Assessment Year
df_prop["N-ValueRatio"] = df_prop["taxvaluedollarcnt"] / df_prop["taxamount"]

# Total Tax Score
df_prop["N-TaxScore"] = df_prop["taxvaluedollarcnt"] * df_prop["taxamount"]

##### Location-related features

In [156]:
# Number of properties per zip code
zip_count = df_prop["regionidzip"].value_counts().to_dict()
df_prop["N-zip_count"] = df_prop["regionidzip"].map(zip_count)

# Number of properties per city
city_count = df_prop["regionidcity"].value_counts().to_dict()
df_prop["N-city_count"] = df_prop["regionidcity"].map(city_count)

# Number of properties per country
region_count = df_prop["regionidcounty"].value_counts().to_dict()
df_prop["N-county_count"] = df_prop["regionidcounty"].map(region_count)

##### Features that are the simplification of other features

In [157]:
# Indicator whether AC is present or not
df_prop["N-ACInd"] = (df_prop["airconditioningtypeid"] != 5) * 1
df_prop["N-ACInd"] = df_prop["N-ACInd"].astype(bool)

# Indicator whether heating is present or not
df_prop["N-HeatInd"] = (df_prop["heatingorsystemtypeid"] != 13) * 1
df_prop["N-HeatInd"] = df_prop["N-HeatInd"].astype(bool)

# Type of land use for which the property is zoned - previously there were 25 categories, now they are compressed to 4
df_prop["N-PropType"] = df_prop["propertylandusetypeid"].replace(
    {
        31: "Mixed",
        46: "Other",
        47: "Mixed",
        246: "Mixed",
        247: "Mixed",
        248: "Mixed",
        260: "Home",
        261: "Home",
        262: "Home",
        263: "Home",
        264: "Home",
        265: "Home",
        266: "Home",
        267: "Home",
        268: "Home",
        269: "Not Built",
        270: "Home",
        271: "Home",
        273: "Home",
        274: "Other",
        275: "Home",
        276: "Home",
        279: "Home",
        290: "Not Built",
        291: "Not Built",
    }
)
df_prop.drop(columns="propertylandusetypeid", inplace=True)

##### Custom features related to 'structuretaxvaluedollarcnt' because it is considered an important specification

In [158]:
# Average structuretaxvaluedollarct per city
group = (
    df_prop.groupby("regionidcity")["structuretaxvaluedollarcnt"]
    .aggregate("mean")
    .to_dict()
)
df_prop["N-Avg-structuretaxvaluedollarcnt"] = df_prop["regionidcity"].map(
    group
)  # assign 'regionidcity' to the mean calculated above and put into a dictionary with key 'regionidcity'

# Deviation of the value from the mean
df_prop["N-Dev-structuretaxvaluedollarcnt"] = (
    abs(
        (
            df_prop["structuretaxvaluedollarcnt"]
            - df_prop["N-Avg-structuretaxvaluedollarcnt"]
        )
    )
    / df_prop["N-Avg-structuretaxvaluedollarcnt"]
)

In [159]:
# In case there are "Infinity" values ​​in the dataset resulting from divisions by zero, these are set to 0 as it is semantically correct.
df_prop.replace([np.inf, -np.inf], 0, inplace=True)

In [ ]:
df_prop.info(max_cols=TOP_K_DISPLAY)
print("Shape: ", df_prop.shape)

<a id='features_selection'></a>
### Features importance, feature selection and preparation of the final dataset


At this point the dataset no longer has missing values ​​and intuitively the multicollinearity problems between the input features, highlighted in the previous analyses, have been resolved. The feature selection process can then be carried out in order to build even more accurate forecasting models.

In [ ]:
cat_var_names = set(
    [
        "airconditioningtypeid",
        "heatingorsystemtypeid",
        "propertycountylandusecode",
        "N-PropType",
        "propertyzoningdesc",
        "regionidcity",
        "regionidcounty",
        "regionidneighborhood",
        "regionidzip",
        "fips",
        "rawcensustractandblock",
    ]
)

for c in cat_var_names:
    print(c, len(df_prop[c].unique()))

However, it is decided to remove some of the categorical variables with many distinct elements (specifically 'propertyzoningdesc', 'propertycountylandusecode', 'regionidneighborhood', 'regionidzip', 'regionidcity', 'rawcensustractandblock') so as not to exponentially increase the size of the matrix after the OneHotEncoding process (also possible curse of dimensionality problem) and increase the time required for training and testing the models.<br>
Please note that this encoding process is necessary for the correct management of categorical features, otherwise they would be interpreted in a numerical or ordinal way.

In [172]:
removed_cat = set(
    [
        "propertyzoningdesc",
        "propertycountylandusecode",
        "regionidneighborhood",
        "regionidzip",
        "regionidcity",
        "rawcensustractandblock",
    ]
)
one_hot_colmuns = list(cat_var_names.difference(removed_cat))


one_hot_enc = OneHotEncoder(sparse_output=False)
one_hot_enc.fit(df_prop[one_hot_colmuns])
one_hot_tranform_name = one_hot_enc.get_feature_names_out(one_hot_colmuns)

df_one_hot = pd.DataFrame(
    one_hot_enc.transform(df_prop[one_hot_colmuns]), columns=one_hot_tranform_name # type: ignore
)

df_prop_drop_cat = df_prop.drop(columns=list(cat_var_names))
df_prop_final = pd.concat(
    [df_prop_drop_cat.reset_index(), df_one_hot.reset_index()], axis=1
)
df_prop_final.drop(columns=["index"], inplace=True)


# Convert date to integer
df_prop_final["yearbuilt"] = df_prop_final["yearbuilt"].dt.year
df_prop_final["assessmentyear"] = df_prop_final["assessmentyear"].dt.year
to_float64_float32(df_prop_final)
to_int64_int32(df_prop_final)
del df_prop, df_one_hot

In [ ]:
df_prop_final.info(max_cols=TOP_K_DISPLAY)
print("Final shape: ", df_prop_final.shape)
df_prop_final.to_parquet(os.path.join(OUTPUT_DATA_DIR, "final_df_2016.parquet"))

In [61]:
X = df_prop_final.drop(columns=["parcelid", "logerror", "transactiondate"])
y = df_prop_final["logerror"]

X and y will contain respectively:
- the features to use for building the model
- the actual values ​​to be used for the supervised learning task

*Please note that the variables only contain data of type float (or convertible to float) because these are the only types accepted by the ML algorithms used*

In the section below, we proceed to the tuning of the "n_estimators" parameter to build a model using gradient boosting that, once trained, will be used to select the features considered to have the greatest impact by the algorithm<br>
It is also noted that the XGBoost library was chosen for its high efficiency and accuracy.

In [ ]:
tuning_params = {"n_estimators": [i for i in range(1, 26, 1)]}

X_train_80, X_test, y_train_80, y_test = train_test_split(X, y, test_size=0.20)

xgb_model = xgb.XGBRegressor()
xgb_grid = GridSearchCV(
    estimator=xgb_model,
    param_grid=tuning_params,
    cv=5,
    scoring="neg_root_mean_squared_error",
    verbose=0,
    n_jobs=-1,
)

print("Tuning XGBoost hyperparameters:")
xgb_grid.fit(X_train_80, y_train_80)
print("Best Score: {:.4f}".format(-xgb_grid.best_score_))
print("Best Params: ", xgb_grid.best_params_)

test_mse = root_mean_squared_error(y_true=y_test, y_pred=xgb_grid.predict(X_test))
print("MSE: {:.4f}".format(test_mse))

In [ ]:
plot_feature_importance(
    xgb_grid.best_estimator_.feature_importances_, X.columns, "XGBoost Model ", limit=40
)

*You can see from the graph regarding the features importance that many of the custom variables added previously have a significant weight in the construction of the model*

The presumed best subset of n features is now chosen using the subsequent feature ranking method with recursive feature elimination using cross-validation sets in order to reduce, as anticipated, the dimensionality of the data frame and the consequent problems of multicollinearity and curse of dimensionality to further optimize the predictions.<br>
It was decided to set a minimum number of 10 features so that the final model does not have too few variables that would consequently not be able to explain the data accurately enough.

In [ ]:
selector = RFECV(
    xgb_grid.best_estimator_,
    step=1,
    cv=5,
    scoring="neg_root_mean_squared_error",
    min_features_to_select=10,
    n_jobs=-1,
)
selector.fit(X, y)
selector.n_features_

The features selected by the algorithm are therefore 12, specifically:

In [ ]:
list_important_features = [
    x[0] for x in list(zip(X.columns, selector.support_)) if x[1]
]
print(list_important_features)

The dataset containing the selected features is created and saved offline so that it can then be used for the construction of the two final models or for any future analyses.

In [ ]:
selected_important_X = X[list_important_features]
selected_important_X.shape

In [68]:
selected_important_X.to_parquet(OUTPUT_DATA_DIR + "final_df_2016_filtered.parquet")

<a id='mod_1'></a>
### Building the model using Random Forest and tuning its parameters

At this stage it was decided to implement parameter tuning in a more sophisticated way.
Specifically, the method with cross validation sets will search for the best combination of "n_estimators", "max_leaf_nodes" and "min_samples_leaf", 3 of the main hyperparameters for the construction of the regressor using the Random Forest ensembling method.

In [69]:
X_train_80, X_test, y_train_80, y_test = train_test_split(
    selected_important_X, y, test_size=0.20
)

In [ ]:
tuning_params = {
    "n_estimators": [i for i in range(10, 81, 10)],
    "max_leaf_nodes": [
        10,
        30,
        50,
        100,
        200,
    ],  # Grow trees with max_leaf_nodes in best-first fashion
    "min_samples_leaf": [i for i in range(1, 5, 1)],
}  # The minimum number of samples required to be at a leaf node.

rf = RandomForestRegressor()
rf_model = GridSearchCV(
    estimator=rf,
    param_grid=tuning_params,
    cv=5,
    scoring="neg_root_mean_squared_error",
    verbose=0,
    n_jobs=-1,
)

print("Tuning Random Forest hyperparameters:")
rf_model.fit(X_train_80, y_train_80)
print("Best Score: {:.4f}".format(-rf_model.best_score_))
print("Best Params: ", rf_model.best_params_)

test_mse = root_mean_squared_error(y_true=y_test, y_pred=rf_model.predict(X_test))
print("MSE: {:.4f}".format(test_mse))

joblib.dump(rf_model, os.path.join(MODEL_DIR, "random_forest_model.pkl"))

It is believed that the Mean Squared Error values ​​obtained, especially in the testing set, are sufficiently low and that consequently the model using Random Forest is quite accurate. Furthermore, using cross validation sets has limited the possibility of overfitting of the predictor

In [ ]:
N_TESTS = 2
step = 2
offset = 10

stats = np.array([])
n_trees = [
    1 if i == 0 else i
    for i in range(0, rf_model.best_params_["n_estimators"] + offset, step)
]

for l in n_trees:
    y_preds = np.array([])

    for i in range(N_TESTS):
        Xs, ys = resample(X_train_80, y_train_80, n_samples=int(0.67 * len(y_train_80)))  # type: ignore

        rf = RandomForestRegressor(
            n_estimators=l,
            max_leaf_nodes=rf_model.best_params_["max_leaf_nodes"],
            min_samples_leaf=rf_model.best_params_["min_samples_leaf"],
            n_jobs=-1,
        )
        rf.fit(Xs, ys)

        y_pred = rf.predict(X_test)
        y_preds = np.column_stack([y_preds, y_pred]) if y_preds.size else y_pred

    dt_bias = (y_test - np.mean(y_preds, axis=1)) ** 2
    dt_variance = np.var(y_preds, axis=1)
    dt_error = (y_preds - y_test.reshape(-1, 1)) ** 2.0

    run_stats = np.array([dt_error.mean(), dt_bias.mean(), dt_variance.mean()])

    stats = np.column_stack([stats, run_stats]) if stats.size else run_stats


fig, ax = plt.subplots(figsize=(8, 8))
ax.set_title("Bias-Variance Decomposition Analysis - Random Forest")
ax.plot(n_trees, stats[0, :], "o:", label="Error")
ax.plot(n_trees, stats[1, :], "o:", label="Bias$^2$")
ax.plot(n_trees, stats[2, :], "o:", label="Variance")
ax.set_xlabel("Number of Trees")
ax.grid()
ax.legend()

print("Error/Bias/Variance at the last iteration:", stats[:, -1])

As can be seen from the graph *number of trees - total error*, the error decreases (and the score improves) as the number of estimators used increases. In particular, it is clear, as per theoretical intuition, that since the predictor is based on a forest of trees, each of these by definition is sought to be fully grown, therefore with low distortion and high variance, and that the latter will then be reduced with the ensemble. In this specific case, it is highlighted how it is possible to significantly reduce the variance already using only 4 estimators.<br>
We then recall that the formula for the decomposition of the total error is:
$$
Error(M) = Bias^2 + Variance + Noise
$$

<a id='mod_2'></a>
### Construction of the model exploiting Gradient Boosting and tuning of its parameters

Even for the construction of this model it was decided to perform a more sophisticated tuning of the parameters.
Specifically, the method with cross validation sets will search for the best combination of "n_estimators", "max_leaves" and "learning_rate", 3 of the main hyperparameters for the construction of the regressor using the Gradient Boosting method.

In [75]:
X_train_80, X_test, y_train_80, y_test = train_test_split(
    selected_important_X, y, test_size=0.20
)

In [ ]:
tuning_params = {
    "n_estimators": [i for i in range(10, 101, 10)],
    "max_leaves": [
        2,
        5,
        10,
        50,
        100,
        200,
    ],  # Maximum number of leaves; 0 indicates no limit
    "learning_rate": [0.1, 0.2, 0.3, 0.4],
}  # Boosting learning rate (xgb's “eta”).
# Step size shrinkage used in update to prevents overfitting. The value must be between 0 and 1. Default is 0.3.

xgb_m = xgb.XGBRegressor()
xgb_model = GridSearchCV(
    estimator=xgb_m,
    param_grid=tuning_params,
    cv=5,
    scoring="neg_root_mean_squared_error",
    verbose=0,
    n_jobs=-1,
)

print("Tuning XGBoost hyperparameters:")
xgb_model.fit(X_train_80, y_train_80)
print("Best Score: {:.4f}".format(-xgb_model.best_score_))
print("Best Params: ", xgb_model.best_params_)

test_mse = root_mean_squared_error(y_true=y_test, y_pred=xgb_model.predict(X_test))
print("MSE: {:.4f}".format(test_mse))

joblib.dump(xgb_model, os.path.join(MODEL_DIR, "xgboost_model.pkl"))

Also in this case it is believed that the Mean Squared Error values ​​obtained, in particular in the testing set, are sufficiently low and that consequently the model based on Gradient Boosting is quite accurate. Furthermore, using the cross validation sets has limited the possibility of overfitting in the regressor

In [ ]:
N_TESTS = 2
step = 2
offset = 10

stats = np.array([])
n_trees = [
    1 if i == 0 else i
    for i in range(0, xgb_model.best_params_["n_estimators"] + offset, step)
]

for l in n_trees:
    y_preds = np.array([])

    for i in range(N_TESTS):
        Xs, ys = resample(X_train_80, y_train_80, n_samples=int(0.67 * len(y_train_80)))  # type: ignore

        xgb_m = xgb.XGBRegressor(
            n_estimators=l,
            learning_rate=xgb_model.best_params_["learning_rate"],
            max_leaves=xgb_model.best_params_["max_leaves"],
            n_jobs=-1,
        )
        xgb_m.fit(Xs, ys)

        y_pred = xgb_m.predict(X_test)
        y_preds = np.column_stack([y_preds, y_pred]) if y_preds.size else y_pred

    dt_bias = (y_test - np.mean(y_preds, axis=1)) ** 2
    dt_variance = np.var(y_preds, axis=1)
    dt_error = (y_preds - y_test.reshape(-1, 1)) ** 2.0

    run_stats = np.array([dt_error.mean(), dt_bias.mean(), dt_variance.mean()])

    stats = np.column_stack([stats, run_stats]) if stats.size else run_stats


fig, ax = plt.subplots(figsize=(8, 8))
ax.set_title("Bias-Variance Decomposition Analysis - XGBoost")
ax.plot(n_trees, stats[0, :], "o:", label="Error")
ax.plot(n_trees, stats[1, :], "o:", label="Bias$^2$")
ax.plot(n_trees, stats[2, :], "o:", label="Variance")
ax.set_xlabel("Number of Trees")
ax.grid()
ax.legend()

print("Error/Bias/Variance at the last iteration:", stats[:, -1])

As can be seen from the graph, also in this case the error decreases (and the score improves) as the number of estimators used increases. In particular, it is clear, as per theoretical intuition, that since the predictor is based on tree boosting, each of these by definition is sought to be small in size, therefore with low variance but high distortion, and that the latter will then be reduced with the ensemble.
In this specific case the decomposition shows a nearly zero variance and an error that decreases exponentially as the distortion decreases, which can also be significantly reduced using only 10 estimators.

<a id='comparison_and_analysis'></a>
### Comparison and analysis of models

Once the construction of the final statistical models has been completed, we now proceed to analyse their behaviour in more depth, in particular by focusing on some of the buildings where the given log error is the worst and others where it is the best.

The buildings with log error between the most extreme value and the most extreme value minus 1 are then extracted (worst items) and ten of the buildings where the log error is zero (best items).

In [ ]:
sns.displot(df_prop_final, x="logerror", height=6, aspect=16 / 9);

The variable 'logerror' follows a normal distribution

In [96]:
offset = 1
max_error_pos = df_prop_final["logerror"].max() - offset
max_error_neg = df_prop_final["logerror"].min() + offset

worst_items = pd.concat(
    [
        df_prop_final[df_prop_final["logerror"] >= max_error_pos],
        df_prop_final[df_prop_final["logerror"] <= max_error_neg],
    ]
)
best_items = df_prop_final[df_prop_final["logerror"] == 0].head(10)

In [ ]:
worst_items_X = selected_important_X.iloc[worst_items.index]
worst_items_X

In [ ]:
best_items_X = selected_important_X.iloc[best_items.index]
best_items_X

In [ ]:
plt.figure(figsize=(8, 6))
plt.bar(
    range(len(worst_items)),
    worst_items["logerror"],
    color=["# 9dd866"],
    label="Target value",
)
plt.bar(
    range(len(worst_items)),
    rf_model.predict(worst_items_X),
    color=["# ffa056"],
    label="RF value",
)
plt.bar(
    range(len(worst_items)),
    xgb_model.predict(worst_items_X),
    color=["# 0b84a5"],
    label="XGB value",
)
plt.legend()
plt.title("Log-errors comparisons - Worst items");

From the bar chart of the predictions on problematic items, it can be noted that the estimates of the two final models created are very far from the target value calculated by the Zestimate estimator. This is probably due to the presence of many missing or outlier values, treated with analysis, recovery and correction techniques different from those used by the Zestimate team.
However, it is assumed a priori that items with a high absolute log error may present anomalies or non-standard values ​​for some features, since the competition estimator itself returns predicted values ​​that are quite distant from the real log(SalePrice).

To verify these hypotheses, it is thought that it is necessary to contact experts in the field of Data Science, the Zestimate team and real estate technicians.

In [ ]:
plt.figure(figsize=(8, 6))
plt.plot(
    range(len(best_items)),
    best_items["logerror"],
    color="# 9dd866",
    label="Target value",
)
plt.plot(
    range(len(best_items)),
    rf_model.predict(best_items_X),
    color="# ffa056",
    label="RF value",
)
plt.plot(
    range(len(best_items)),
    xgb_model.predict(best_items_X),
    color="# 0b84a5",
    label="XGB value",
)
plt.legend()
plt.title("Log-errors comparisons - Best Items");

In the line chart relating to the forecasts on items whose log error is zero, the two models created also return satisfactory values: in fact, the deviation from the target value of Zestimate is in the order of cents.

In general, even in the graphs relating to best items and worst items, no big differences in terms of accuracy are noted between the model with Random Forest and the one with Gradient Boosting, even if the MSE value obtained in the testing set of the first model is slightly lower than the one returned by the second.

<a id='final'></a>
### Final considerations

The various steps carried out are then briefly summarized:
- Dataset analysis, feature engineering and missing value management
- Features selection and model construction
- Analysis and comparison of the obtained models

In conclusion, thanks to this task it was possible to test on a real, extended and complex dataset, the performances of some Machine Learning methods such as Random Forest and Gradient Boosting, both exploiting decision trees but using different intuitions.<br>
However, as previously mentioned, there is not enough evidence to prefer one model over the other since both return accurate and efficient predictions and the deviation in the value of the evaluation metric for the testing set is very low.
Generally, predictors that use the Gradient Boosting technique perform better than those that exploit Random Forest by definition since they have as their objective the minimization of a loss function and the characteristic of additively building the various trees, even if they are more prone to overfitting.<br>
Probably, with a deeper knowledge of the data, of their meaning, of their real trend and with the addition of external information to the dataset it is possible to obtain even more precise results and to build the optimal regression model.

##### *Summary table containing the evaluation data*
